<a href="https://colab.research.google.com/github/DovTech/models_roofs_experiments/blob/main/mscoco_to_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import json
import os
import gdown
import shutil
from PIL import Image

# Files load

In [ ]:
gdown.download("https://drive.google.com/file/d/1KrbT2KbayuNtsACy7GHmGbUWtikpbSfI/view?usp=sharing", fuzzy=True) #train_small (img + msk)
gdown.download("https://drive.google.com/file/d/1MpYdtlbfdCoKW1K1qTcqA_en6RnBtb6g/view?usp=drive_link", fuzzy=True) #val_small (img + msk)
#gdown.download("https://drive.google.com/file/d/1qu98Ty75AdL-zC0l-4MFQPidSTCROEoL/view?usp=drive_link", fuzzy=True) #test_images (img)
gdown.download("https://drive.google.com/file/d/1OVJLngs_my3BJNks95-VifrHv-xdIxPW/view?usp=sharing", fuzzy=True) #test_img_msk (img + msk) [created by train dataset]

gdown.download("https://drive.google.com/file/d/1a1ZmzO6ELZt_AfPa5PwuGrC1OEebwyp1/view?usp=sharing", fuzzy=True) #train_annotation_small
gdown.download("https://drive.google.com/file/d/1V_FNruO6Oz35E9FC_ODj1lq3d4ZoAE2Y/view?usp=sharing", fuzzy=True) #val_annotations_small

!unzip /content/train_small.zip
!unzip /content/val_small.zip
# !tar -xvf /content/test_images.tar.gz
!unzip /content/test_img_msk.zip

Downloading...
From: https://drive.google.com/uc?id=1KrbT2KbayuNtsACy7GHmGbUWtikpbSfI
To: /content/train_small.zip
100%|██████████| 210M/210M [00:02<00:00, 70.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MpYdtlbfdCoKW1K1qTcqA_en6RnBtb6g
To: /content/val_small.zip
100%|██████████| 91.8M/91.8M [00:00<00:00, 190MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OVJLngs_my3BJNks95-VifrHv-xdIxPW
To: /content/test_img_msk.zip
100%|██████████| 3.84M/3.84M [00:00<00:00, 140MB/s]
Downloading...
From: https://drive.google.com/uc?id=1a1ZmzO6ELZt_AfPa5PwuGrC1OEebwyp1
To: /content/train_annotation_small.json
100%|██████████| 21.3M/21.3M [00:00<00:00, 186MB/s]
Downloading...
From: https://drive.google.com/uc?id=1V_FNruO6Oz35E9FC_ODj1lq3d4ZoAE2Y
To: /content/val_annotation_small.json
100%|██████████| 4.61M/4.61M [00:00<00:00, 39.9MB/s]


Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: content/val_small/masks/000000015133_mask.jpg  
  inflating: content/val_small/masks/000000036218_mask.jpg  
  inflating: content/val_small/masks/000000048332_mask.jpg  
  inflating: content/val_small/masks/000000037706_mask.jpg  
  inflating: content/val_small/masks/000000006237_mask.jpg  
  inflating: content/val_small/masks/000000050357_mask.jpg  
  inflating: content/val_small/masks/000000031921_mask.jpg  
  inflating: content/val_small/masks/000000034652_mask.jpg  
  inflating: content/val_small/masks/000000035483_mask.jpg  
  inflating: content/val_small/masks/000000051993_mask.jpg  
  inflating: content/val_small/masks/000000000108_mask.jpg  
  inflating: content/val_small/masks/000000046902_mask.jpg  
  inflating: content/val_small/masks/000000029294_mask.jpg  
  inflating: content/val_small/masks/000000052964_mask.jpg  
  inflating: content/val_small/masks/000000036788_mask.jpg  
  inflating: cont

# Pathes

In [ ]:
!mkdir /content/yolo_format

In [ ]:
DATA_DIR = '/content'

img_train_dir = os.path.join(DATA_DIR, 'train_small/images/')
mask_train_dir = os.path.join(DATA_DIR, 'train_small/masks/')

img_valid_dir = os.path.join(DATA_DIR, 'val_small/images/')
mask_valid_dir = os.path.join(DATA_DIR, 'val_small/masks/')

img_test_dir = os.path.join(DATA_DIR, 'test_images/')

train_annot = os.path.join(DATA_DIR, 'train_annotation_small.json')
val_annot = os.path.join(DATA_DIR, 'val_annotation_small.json')

# Create datasets

In [ ]:
def yolov8_format_creator(path, dirname):
    yolo_format = '/content/yolo_format/' + dirname + '/'
    os.mkdir(yolo_format)
    with open(path, 'r') as f:
        data = json.load(f)
        df = pd.DataFrame(data['annotations']).drop(['area', 'bbox', 'iscrowd'], axis=1)
        df = df.sort_values(by='image_id')

    img_ids = df['image_id'].unique()
    for id in img_ids:
        yolo_lines = []
        segs = df[df['image_id'] == id]['segmentation']
        #cats = df[df['image_id'] == id]['category_id']
        #for seg, cat in zip(segs, cats):
        for seg in segs:
            seg = [coord / 300 for coord in seg[0]]
            seg.insert(0, 0)
            yolo_lines += "".join(str(x) + ' ' for x in seg) + '\n'
        filename = str(id).rjust(12, '0')
        with open(f'{yolo_format}{filename}.txt', 'w') as f:
            f.writelines(yolo_lines)
    zip_dst = f'/content/drive/MyDrive/roofs_task/{dirname}'
    shutil.make_archive(zip_dst, 'zip', yolo_format)

In [ ]:
!rm -r /content/yolo_format/train_yolo

In [ ]:
yolov8_format_creator(train_annot, 'train_yolo')

In [ ]:
yolov8_format_creator(val_annot, 'val_yolo')